# Preparing basic tools

In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

If the image is 8-bit unsigned, it is displayed as is.
<br>If the image is 16-bit unsigned or 32-bit integer, the pixels are divided by 256. That is, the value range [0,255*256] is mapped to [0,255].
<br>If the image is 32-bit floating-point, the pixel values are multiplied by 255. **That is, the value range [0,1] is mapped to [0,255].**

In [2]:
def read_image(path : str):
    image = cv.imread(path)
    image //= 255 # see above
    grey_img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    return grey_img, np.shape(grey_img)

In [3]:
def identify_classes(X):
    return np.unique(X)

In [4]:
def display_image(title:str, image):
    cv.imshow(title, image)
    cv.waitKey()
    cv.destroyAllWindows()

In [82]:
def gauss_noise(X, m, n, cl1, cl2, m1, sig1, m2, sig2):
    gaussian_class_1 = np.random.normal(m1, sig1, (m,n))
    gaussian_class_1[X == cl2] = 0
    gaussian_class_2 = np.random.normal(m2, sig2, (m,n)) # will probably have to use np.where class == {1,2}
    gaussian_class_2[X == cl1] = 0
    noise = gaussian_class_1 + gaussian_class_2

    return noise

In [6]:
def error_rate(tolerance, A, B, m, n):
    return 1 - ((abs(A - B) <= tolerance).sum())/(m*n)

In [45]:
def evaluate_kmeans(k, A, B):
    shape = A.shape
    
    model = KMeans(n_clusters=k, random_state=0).fit(B.reshape(-1,1))
    values = model.cluster_centers_
    labels = model.labels_
    img_seg = values[labels].reshape(shape)
    
    return img_seg, error_rate(0.1, img_seg, A, shape[0], shape[1])

# Puisqu'il s'agit de segmenter ... segmentons !

In [46]:
path = './images_BW/cible2.bmp'
image, shape = read_image(path)
cl1, cl2 = identify_classes(image)
print(f'classe 1 : {cl1}, classe 2 : {cl2}')

classe 1 : 0, classe 2 : 1


In [47]:
display_image('image', 255*image)

---

In [64]:
noisy = (image + gauss_noise(image, shape[0], shape[1],cl1, cl2, 0, 1, 0, 0)).clip(0,1)
noisy_1 = (noisy + gauss_noise(image, shape[0], shape[1],cl1, cl2, 0, 0, 0, 1)).clip(0,1)
noice = [noisy, noisy_1]
combine_noisy = np.concatenate(tuple(noice), axis=1)

display_image('Noisy, Noisy_1', combine_noisy)

In [66]:
img_seg, error = evaluate_kmeans(2, image, noisy_1)

In [67]:
error

0.52252197265625

In [68]:
display_image('Noisy image, KMeans, Image', np.concatenate((noisy_1, img_seg, image), axis=1))

---

In [69]:
from scipy.stats import norm

In [75]:
def calc_prior(X, m, n, cl1, cl2):
    return ((X == cl1).sum()) / (m*n), ((X == cl2).sum()) / (m*n) # p(cl1), p(cl2) estimates

In [74]:
def MPM_Gauss(Y, cl1, cl2, p1, p2, m1, sig1, m2, sig2):
    flat_Y = Y.reshape(-1,1)
    Y_hat = np.array([cl1 if p1*norm(m1, sig1).pdf(y-cl1) > p2*norm(m2, sig2).pdf(y-cl2) else cl2 for y in flat_Y])
    
    return Y_hat.reshape(Y.shape)

In [88]:
X = image
cl1, cl2 = identify_classes(X)
print(f'Classe 1 : {cl1} (black)\nClasse 2 : {cl2} (white)')

Classe 1 : 0 (black)
Classe 2 : 1 (white)


In [80]:
m, n = X.shape[0], X.shape[1]

In [86]:
p1, p2 = calc_prior(X, m, n, cl1, cl2)
print(f'p(cl1) = p1 = {p1}\np(cl2) = p2 = {p2}')

p(cl1) = p1 = 0.69488525390625
p(cl2) = p2 = 0.30511474609375


In [90]:
m1, m2, sig1, sig2 = 0, 0, .3, .3

In [92]:
Y = image + gauss_noise(X, m, n, cl1, cl2, m1, sig1, m2, sig2)
display_image('Y', Y)

In [94]:
Y_hat = MPM_Gauss(Y, cl1, cl2, p1, p2, m1, sig1, m2, sig2)

In [96]:
display_image('Y_hat', Y_hat*255)

In [97]:
print(error_rate(0, X, Y_hat, m, n))

0.0433197021484375


In [98]:
print(evaluate_kmeans(2, X, Y)[1])

0.053192138671875
